In [7]:
import pandas as pd
import plotly.express as px
from pathlib import Path

# Define paths
ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data" / "processed"
ANALYTICS_PATH = DATA_DIR / "analytics_table.parquet"

# Load data
df = pd.read_parquet(ANALYTICS_PATH)
print(f"Loaded {len(df)} rows from {ANALYTICS_PATH}")
df.head()

Loaded 5 rows from /home/azoz-laptop/Desktop/Git-Projects/W2/data/processed/analytics_table.parquet


,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,...,hour,country,signup_date,date_user,year_user,month_user,dow_user,hour_user,amount_winsor,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,...,10.0,SA,2025-11-15 00:00:00+00:00,2025-11-15,2025,2025-11,Saturday,0,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,...,11.0,SA,2025-11-20 00:00:00+00:00,2025-11-20,2025,2025-11,Thursday,0,8.135,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,...,9.0,AE,2025-11-22 00:00:00+00:00,2025-11-22,2025,2025-11,Saturday,0,<NA>,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,...,14.0,SA,2025-11-15 00:00:00+00:00,2025-11-15,2025,2025-11,Saturday,0,25.0,False
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,...,NaN,SA,2025-11-25 00:00:00+00:00,2025-11-25,2025,2025-11,Tuesday,0,97.75,True


## Data Overview
Check data types and missing values.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   order_id            5 non-null      object             
 1   user_id             5 non-null      object             
 2   amount              4 non-null      Float64            
 3   quantity            4 non-null      Int64              
 4   created_at          4 non-null      datetime64[ns, UTC]
 5   status              5 non-null      string             
 6   status_clean        5 non-null      object             
 7   amount__isna        5 non-null      bool               
 8   quantity__isna      5 non-null      bool               
 9   date                4 non-null      object             
 10  year                4 non-null      float64            
 11  month               4 non-null      string             
 12  dow                 4 non-null      obje

## Analysis

### 1. Revenue by Country

In [9]:
# Summary Table
revenue_summary = (
    df.groupby("country")
    .agg(
        n_orders=("order_id", "count"),
        total_revenue=("amount", "sum"),
        avg_revenue=("amount", "mean"),
        median_revenue=("amount", "median")
    )
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)
print("Revenue Summary by Country:")
print(revenue_summary)

# Chart
fig = px.bar(revenue_summary, x="country", y="total_revenue", title="Total Revenue by Country", text_auto=True)
fig.update_layout(title_x=0.5)
fig.show()

# Save figure
FIGURES_DIR = ROOT / "reports" / "figures"
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
fig.write_image(FIGURES_DIR / "revenue_by_country.png")
fig.write_html(FIGURES_DIR / "revenue_by_country.html")

Revenue Summary by Country:
  country  n_orders  total_revenue  avg_revenue  median_revenue
1      SA         4          145.5       36.375           18.75
0      AE         1            0.0         <NA>            <NA>


**Interpretation:**
- The table shows the total revenue, average, and median revenue per country.
- `n_orders` indicates the sample size for each group.
- The bar chart visualizes the total revenue contribution.

### 2. Order Status Distribution

In [10]:
# Summary Table
status_summary = (
    df["status_clean"]
    .value_counts(normalize=True)
    .reset_index()
    .rename(columns={"proportion": "rate", "status_clean": "status"})
)
status_summary["count"] = df["status_clean"].value_counts().values
print("Order Status Distribution:")
print(status_summary)

# Chart
fig = px.pie(df, names="status_clean", title="Order Status Distribution")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Save
fig.write_image(FIGURES_DIR / "status_distribution.png")
fig.write_html(FIGURES_DIR / "status_distribution.html")

Order Status Distribution:
   status  rate  count
0    paid   0.8      4
1  refund   0.2      1


**Interpretation:**
- The pie chart and table show the proportion of orders in each status.
- This helps identify the completion rate vs. refund rate.

### 3. Amount Distribution (Outliers)

In [11]:
# Summary Stats
print("Amount Statistics:")
print(df["amount"].describe())

# Chart
fig = px.histogram(df, x="amount", title="Distribution of Order Amounts", nbins=20, marginal="box")
fig.update_layout(title_x=0.5)
fig.show()

# Save
fig.write_image(FIGURES_DIR / "amount_distribution.png")
fig.write_html(FIGURES_DIR / "amount_distribution.html")

Amount Statistics:
count          4.0
mean        36.375
std      43.022039
min            8.0
25%         11.375
50%          18.75
75%          43.75
max          100.0
Name: amount, dtype: Float64


**Interpretation:**
- The histogram shows the spread of order amounts.
- The box plot (marginal) highlights the median and any potential outliers.

### 4. Bootstrap Uncertainty
We use bootstrapping to estimate the confidence interval for the difference in means between groups (if applicable).

In [12]:
import numpy as np

def bootstrap_diff_means(s1, s2, n_boot=1000, seed=42):
    rng = np.random.default_rng(seed)
    # Ensure inputs are numpy arrays and drop NaNs
    s1_arr = np.array(s1)
    s2_arr = np.array(s2)
    s1_arr = s1_arr[~np.isnan(s1_arr)]
    s2_arr = s2_arr[~np.isnan(s2_arr)]
    
    if len(s1_arr) == 0 or len(s2_arr) == 0:
        return {
            "observed_diff": np.nan,
            "ci_lower": np.nan,
            "ci_upper": np.nan,
            "n_boot": n_boot
        }

    observed_diff = s1_arr.mean() - s2_arr.mean()
    
    # Bootstrapping
    boot_diffs = []
    for _ in range(n_boot):
        s1_boot = rng.choice(s1_arr, size=len(s1_arr), replace=True)
        s2_boot = rng.choice(s2_arr, size=len(s2_arr), replace=True)
        boot_diffs.append(s1_boot.mean() - s2_boot.mean())
        
    ci_lower, ci_upper = np.percentile(boot_diffs, [2.5, 97.5])
    
    return {
        "observed_diff": observed_diff,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper,
        "n_boot": n_boot
    }

# Example: Compare amount between two most frequent countries
top_countries = df["country"].value_counts().head(2).index.tolist()

if len(top_countries) >= 2:
    c1, c2 = top_countries[0], top_countries[1]
    # Drop NaNs explicitly before passing
    s1 = df[df["country"] == c1]["amount"].dropna()
    s2 = df[df["country"] == c2]["amount"].dropna()
    
    if len(s1) > 0 and len(s2) > 0:
        result = bootstrap_diff_means(s1, s2)
        
        if np.isnan(result['observed_diff']):
             print(f"Bootstrap Comparison: {c1} vs {c2} (Mean Amount)")
             print("Could not calculate difference (likely empty data after cleaning).")
        else:
            print(f"Bootstrap Comparison: {c1} vs {c2} (Mean Amount)")
            print(f"Observed Difference: {result['observed_diff']:.2f}")
            print(f"95% CI: [{result['ci_lower']:.2f}, {result['ci_upper']:.2f}]")
            
            if result['ci_lower'] > 0 or result['ci_upper'] < 0:
                print("Result: The difference is statistically significant (CI does not cross 0).")
            else:
                print("Result: The difference is NOT statistically significant (CI crosses 0).")
    else:
        print(f"Not enough valid data for {c1} or {c2}.")
else:
    print("Not enough countries to perform comparison.")

Not enough valid data for SA or AE.
